## Training

In [41]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, PowerTransformer, power_transform, scale, StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

### Process Data

In [42]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
OXFORD_DATA_FILE = 'data/OxCGRT_latest.csv'
SOCIAL_EXPLORER_DATA_FILE = 'data/social_explorer_data.csv'
LODES_DATA_FILE = 'data/lodes_us_states_rac_S000_JT00_2018.csv'

In [43]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, OXFORD_DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x1d72d5cf220>)

In [44]:
# Load historical data from local file
oxford_data = pd.read_csv(OXFORD_DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [45]:
oxford_data.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [46]:
def transform_data(data):
    # pipe = make_pipeline(StandardScaler(with_std=False), PowerTransformer(standardize=True))
    pipe = make_pipeline(StandardScaler(with_std=True))
    if len(data.shape) == 1:
        transformed_data = pipe.fit_transform(data.reshape(-1,1))
        return (transformed_data, pipe)
    transformed_data = pipe.fit_transform(data)
    return (transformed_data, pipe)
def apply_transform(pipe, data):
    if len(data.shape) == 1:
        return pipe.transform(data.reshape(-1,1))
    return pipe.transform(data)
def inverse_transform(pipe, data):
    if len(data.shape) == 1:
        return pipe.inverse_transform(data.reshape(-1,1))
    return pipe.inverse_transform(data)

# Load Census Data

In [47]:
census_data = pd.read_csv(SOCIAL_EXPLORER_DATA_FILE, skiprows=[1])
census_data.drop(columns=['FIPS'], inplace=True)
cols = census_data.columns.values
for feat in cols:
    if 'Employed' in feat or 'Unemployed' in feat:
        census_data.drop(columns=[feat], inplace=True)
    
print(census_data.shape)
# Standardize Census data
census_columns = census_data.columns[1:]
census_regions = census_data['Qualifying Name']
census_data, census_pipe = transform_data(census_data.iloc[:, 1:].values)
census_data = pd.DataFrame(data=census_data,columns=census_columns)
census_data.insert(0, 'Qualifying Name', census_regions)
census_data

(52, 67)


,Qualifying Name,Population Density (Per Sq. Mile),Total Population: Under 5 Years,Total Population: 5 to 9 Years,Total Population: 10 to 14 Years,Total Population: 15 to 17 Years,Total Population: 18 to 24 Years,Total Population: 25 to 34 Years,Total Population: 35 to 44 Years,Total Population: 45 to 54 Years,...,Two or More Races Population for Whom Poverty Status Is Determined: Income At or Above Poverty Level,Hispanic or Latino Population for Whom Poverty Status Is Determined: Income Below Poverty Level,Hispanic or Latino Population for Whom Poverty Status Is Determined: Income At or Above Poverty Level,"White Alone, Not Hispanic or Latino Population for Whom Poverty Status Is Determined: Income Below Poverty Level","White Alone, Not Hispanic or Latino Population for Whom Poverty Status Is Determined: Income At or Above Poverty Level",Average Commute to Work (In Min),Total: No Health Insurance Coverage,Total: with Health Insurance Coverage,Total: with Health Insurance Coverage: Public Health Coverage,Total: with Health Insurance Coverage: Private Health Insurance
0,Alabama,-0.215175,-0.199199,-0.198783,-0.202996,-0.187576,-0.200966,-0.224865,-0.213689,-0.198687,...,-0.406229,-0.357922,-0.370031,0.035764,-0.196694,0.019211,-0.110557,-0.208855,-0.169742,-0.212038
1,Alaska,-0.276398,-0.731929,-0.747412,-0.756276,-0.766335,-0.770234,-0.719084,-0.752198,-0.781314,...,-0.479267,-0.466403,-0.416792,-1.045982,-0.986194,-1.479266,-0.555751,-0.795989,-0.777952,-0.821000
2,Arizona,-0.237248,0.113359,0.131618,0.138640,0.123550,0.127765,0.086748,0.075798,0.032126,...,0.169891,0.432408,0.366835,0.064666,0.019404,0.268957,0.217436,0.087234,0.193458,0.039680
3,Arkansas,-0.240081,-0.429100,-0.429038,-0.441630,-0.442853,-0.459233,-0.459413,-0.457708,-0.479107,...,-0.444682,-0.366952,-0.360050,-0.226629,-0.510051,-0.730027,-0.366309,-0.468162,-0.365652,-0.526632
4,California,-0.114776,4.592685,4.590534,4.595037,4.565413,4.699726,4.854253,4.721176,4.605377,...,6.028452,4.723322,5.462247,3.044525,3.249101,1.267942,2.922548,4.762437,4.959829,4.513128
5,Colorado,-0.242339,-0.108516,-0.083428,-0.096046,-0.112476,-0.105672,-0.002302,-0.034050,-0.111759,...,0.019515,-0.102284,0.032445,-0.202310,0.034366,0.268957,-0.156283,-0.089642,-0.165334,-0.065757
6,Connecticut,0.198583,-0.442039,-0.426549,-0.391805,-0.369005,-0.364064,-0.411838,-0.397454,-0.332239,...,-0.296828,-0.235720,-0.230061,-0.704486,-0.389156,0.518704,-0.443696,-0.366942,-0.379466,-0.369035
7,Delaware,0.039405,-0.728392,-0.745281,-0.733108,-0.745704,-0.752348,-0.711181,-0.733769,-0.745239,...,-0.605478,-0.440729,-0.405878,-1.000125,-0.942492,0.268957,-0.609088,-0.752938,-0.728092,-0.774127
8,Washington DC,7.025374,-0.750128,-0.788404,-0.799329,-0.814012,-0.766483,-0.678538,-0.742651,-0.796981,...,-0.617370,-0.459815,-0.407544,-1.099751,-1.047431,1.517688,-0.644596,-0.789312,-0.773587,-0.817492
9,Florida,-0.026119,1.654703,1.638124,1.676872,1.707664,1.703559,1.756218,1.851359,2.068959,...,1.234245,1.350818,1.627017,2.414391,2.211728,0.768450,2.579642,1.937099,2.142770,1.911235


# Load LODES Data

In [48]:
lodes_data = pd.read_csv(LODES_DATA_FILE, skiprows=[1])
print(lodes_data.shape)

# Standardize lodes data
lodes_columns = lodes_data.columns[1:]
lodes_regions = lodes_data['Qualifying Name']
lodes_data, lodes_pipe = transform_data(lodes_data.iloc[:, 1:].values)
lodes_data = pd.DataFrame(data=lodes_data,columns=lodes_columns)
lodes_data.insert(0, 'Qualifying Name', lodes_regions)
lodes_data

(53, 42)


,Qualifying Name,Total number of jobs,Number of jobs for workers age 29 or younger,Number of jobs for workers age 30 to 54,Number of jobs for workers age 55 or older,Number of jobs with earnings $1250/month or less,Number of jobs with earnings $1251/month to $3333/month,Number of jobs with earnings greater than $3333/month,"Number of jobs in NAICS sector 11 (Agriculture, Forestry, Fishing and Hunting)","Number of jobs in NAICS sector 21 (Mining, Quarrying, and Oil and Gas Extraction)",...,Number of jobs for workers with Race: Native Hawaiian or Other Pacific Islander Alone,Number of jobs for workers with Race: Two or More Race Groups,Number of jobs for workers with Ethnicity: Not Hispanic or Latino,Number of jobs for workers with Ethnicity: Hispanic or Latino,Number of jobs for workers with Educational Attainment: Less than high school,"Number of jobs for workers with Educational Attainment: High school or equivalent, no college",Number of jobs for workers with Educational Attainment: Some college or Associate degree,Number of jobs for workers with Educational Attainment: Bachelor's degree or advanced degree,Number of jobs for workers with Sex: Male,Number of jobs for workers with Sex: Female
0,Alabama,-0.237345,-0.213936,-0.230412,-0.275773,-0.209768,-0.164441,-0.295561,-0.200205,-0.184150,...,-0.341177,-0.411469,-0.173293,-0.353288,-0.217136,-0.172233,-0.220376,-0.330595,-0.234360,-0.240236
1,Alaska,-0.865824,-0.883594,-0.850494,-0.881100,-0.901063,-0.876663,-0.834250,-0.438367,-0.378277,...,-0.438624,-0.668772,-1.006028,-0.420129,-0.665779,-0.960720,-0.890574,-0.827994,-0.856639,-0.874624
2,Arizona,0.031355,0.034509,0.040920,0.005494,-0.015539,0.155554,-0.030617,0.110616,0.011710,...,0.150798,0.151446,-0.134219,0.411710,0.181263,0.004365,0.074060,-0.081960,0.056394,0.005730
3,Arkansas,-0.483416,-0.472554,-0.475204,-0.511537,-0.482123,-0.398271,-0.535616,-0.195300,-0.232225,...,-0.254687,-0.454816,-0.516727,-0.339053,-0.380560,-0.472457,-0.488371,-0.537009,-0.482415,-0.484110
4,California,4.583052,4.440443,4.657457,4.528271,4.390194,4.474175,4.694442,6.767279,0.209085,...,5.925422,6.075217,3.895380,5.545917,5.173131,3.843461,4.516446,4.838509,4.657068,4.504238
5,Colorado,-0.024955,-0.017634,-0.011349,-0.064240,-0.078382,-0.061936,0.023528,-0.075460,0.491041,...,-0.081241,0.083557,-0.056035,0.050716,-0.074049,-0.086525,-0.022939,0.044156,0.003350,-0.053880
6,Connecticut,-0.326978,-0.373062,-0.337227,-0.256122,-0.382184,-0.433348,-0.228592,-0.351246,-0.358849,...,-0.302634,-0.318238,-0.364113,-0.195402,-0.320347,-0.410905,-0.356075,-0.176200,-0.340506,-0.312922
7,Delaware,-0.727732,-0.744303,-0.717359,-0.733023,-0.751390,-0.743141,-0.699655,-0.389317,-0.374781,...,-0.415778,-0.576237,-0.831317,-0.386249,-0.571932,-0.803852,-0.749836,-0.688739,-0.725530,-0.729487
8,Washington DC,-0.765744,-0.766056,-0.748090,-0.804075,-0.828186,-0.802563,-0.705878,-0.438503,-0.378308,...,-0.414301,-0.572693,-0.881802,-0.390011,-0.588858,-0.871125,-0.810949,-0.706741,-0.763591,-0.767421
9,Florida,1.895515,1.815707,1.863420,2.042085,2.066848,2.434878,1.442371,1.001738,-0.228018,...,0.353152,1.157849,1.880115,1.668710,1.910242,2.157473,1.994890,1.577006,1.864976,1.925447


In [49]:
# For testing, restrict training data to that before a hypothetical predictor submission date
SUBMISSION_DATE = np.datetime64("2020-11-30")
oxford_data = oxford_data[oxford_data.Date <= SUBMISSION_DATE]

# Remove current day and previous day because data may be incomplete
# oxford_data = oxford_data.groupby("GeoID", as_index=False).apply(lambda x: x.iloc[:-2])

In [50]:
cols = oxford_data.columns
for col in cols:
    if 'Index' in col or 'Flag' in col or 'Wildcard' in col:
        oxford_data.drop(columns=col, inplace=True)

for idx, row in oxford_data[oxford_data['Date'] == '2020-01-01'].iterrows():
    for elem in row[6:].items():
        if (elem[1] == 0.0):
            continue
        if (np.isnan(elem[1])):
            oxford_data.loc[idx, elem[0]] = 0
        
oxford_data['GeoID'] = oxford_data['CountryName'] + '__' + oxford_data['RegionName'].astype(str)
oxford_data['ConfirmedCases'].fillna(0, inplace=True)
oxford_data['ConfirmedDeaths'].fillna(0, inplace=True)
oxford_data['E3_Fiscal measures'].fillna(0, inplace=True)
oxford_data['E4_International support'].fillna(0, inplace=True)
oxford_data['H4_Emergency investment in healthcare'].fillna(0, inplace=True)
oxford_data['H5_Investment in vaccines'].fillna(0, inplace=True)

def log_data(series):
    df = np.log(series)
    df.replace(-np.inf, 0, inplace=True)
    return df
# Transform non categorical features by taking the natural log
oxford_data['E3_Fiscal measures'] = log_data(oxford_data['E3_Fiscal measures'])
oxford_data['E4_International support'] = log_data(oxford_data['E4_International support'])
oxford_data['H4_Emergency investment in healthcare'] = log_data(oxford_data['H4_Emergency investment in healthcare'])
oxford_data['H5_Investment in vaccines'] = log_data(oxford_data['H5_Investment in vaccines'])

new_oxford = []
# Forward fill for each specific region to handle nan for categorical features
for geoid in oxford_data['GeoID'].unique():
    region_df = oxford_data[oxford_data['GeoID'] == geoid]
    for col in region_df.columns[4:].values:
        region_df[col] = region_df[col].ffill()
    new_oxford.append(region_df)
oxford_data = pd.concat(new_oxford)

oxford_data['NewCases'] = oxford_data.groupby('GeoID').ConfirmedCases.diff().fillna(0)
oxford_data['NewDeaths'] = oxford_data.groupby('GeoID').ConfirmedDeaths.diff().fillna(0)
oxford_data.drop(columns=['Jurisdiction', 'RegionCode'], inplace=True)
oxford_data.drop(columns=['CountryCode'], inplace=True)

# move column to be earlier
geoid = oxford_data['GeoID']
oxford_data.drop(labels=['GeoID'], axis = 1, inplace = True)
oxford_data.insert(3, 'GeoID', geoid)

oxford_data.drop(columns=['ConfirmedCases', 'ConfirmedDeaths', 'NewDeaths'], inplace=True)
oxford_data.head()

c:\users\kaush\onedrive\desktop\usc classes\cs499\usc-xprize-covid\xprize\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\users\kaush\onedrive\desktop\usc classes\cs499\usc-xprize-covid\xprize\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-50-c49e8286d7dc>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_df[col] = region_df[col].ffill()


,CountryName,RegionName,Date,GeoID,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,...,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,H6_Facial Coverings,H7_Vaccination policy,NewCases
0,Aruba,NaN,2020-01-01,Aruba__nan,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,2020-01-02,Aruba__nan,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,2020-01-03,Aruba__nan,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,2020-01-04,Aruba__nan,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,2020-01-05,Aruba__nan,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols_categorical = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']

# To be used later
census_cols = list(census_data.columns[1:].values)
lodes_cols = list(lodes_data.columns[1:].values)

In [52]:
npi_cols = []

for col in npi_cols_categorical:
  one_hot = pd.get_dummies(oxford_data[col],prefix=col)
  for new_col in one_hot.columns.values:
    npi_cols.append(new_col)
  oxford_data = oxford_data.drop(col, axis=1)
  oxford_data = oxford_data.join(one_hot)
npi_cols

['C1_School closing_0.0',
 'C1_School closing_1.0',
 'C1_School closing_2.0',
 'C1_School closing_3.0',
 'C2_Workplace closing_0.0',
 'C2_Workplace closing_1.0',
 'C2_Workplace closing_2.0',
 'C2_Workplace closing_3.0',
 'C3_Cancel public events_0.0',
 'C3_Cancel public events_1.0',
 'C3_Cancel public events_2.0',
 'C4_Restrictions on gatherings_0.0',
 'C4_Restrictions on gatherings_1.0',
 'C4_Restrictions on gatherings_2.0',
 'C4_Restrictions on gatherings_3.0',
 'C4_Restrictions on gatherings_4.0',
 'C5_Close public transport_0.0',
 'C5_Close public transport_1.0',
 'C5_Close public transport_2.0',
 'C6_Stay at home requirements_0.0',
 'C6_Stay at home requirements_1.0',
 'C6_Stay at home requirements_2.0',
 'C6_Stay at home requirements_3.0',
 'C7_Restrictions on internal movement_0.0',
 'C7_Restrictions on internal movement_1.0',
 'C7_Restrictions on internal movement_2.0',
 'C8_International travel controls_0.0',
 'C8_International travel controls_1.0',
 'C8_International travel c

In [53]:
oxford_data = oxford_data[id_cols + npi_cols + cases_col]

oxford_data = oxford_data.sort_values(by=['Date', 'GeoID'])
# Move columns around
date_col = oxford_data.pop('Date')
oxford_data.insert(0, 'Date', date_col)
# cases_col = oxford_data.pop('NewCases')
# oxford_data.insert(4, 'NewCases', cases_col)

oxford_data = oxford_data.reset_index(drop=True)
oxford_data

# california_df = oxford_data[oxford_data['RegionName'] == 'California']
# plt.bar(california_df['Date'], california_df['NewCases'])

# texas_df = oxford_data[oxford_data['RegionName'] == 'Texas']
# plt.bar(texas_df['Date'], texas_df['NewCases'])

,Date,CountryName,RegionName,GeoID,C1_School closing_0.0,C1_School closing_1.0,C1_School closing_2.0,C1_School closing_3.0,C2_Workplace closing_0.0,C2_Workplace closing_1.0,...,H2_Testing policy_3.0,H3_Contact tracing_0.0,H3_Contact tracing_1.0,H3_Contact tracing_2.0,H6_Facial Coverings_0.0,H6_Facial Coverings_1.0,H6_Facial Coverings_2.0,H6_Facial Coverings_3.0,H6_Facial Coverings_4.0,NewCases
0,2020-01-01,Afghanistan,NaN,Afghanistan__nan,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
1,2020-01-01,Albania,NaN,Albania__nan,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
2,2020-01-01,Algeria,NaN,Algeria__nan,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
3,2020-01-01,Andorra,NaN,Andorra__nan,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
4,2020-01-01,Angola,NaN,Angola__nan,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93795,2020-11-30,Venezuela,NaN,Venezuela__nan,0,0,0,1,0,0,...,1,1,0,0,0,0,0,0,1,354.0
93796,2020-11-30,Vietnam,NaN,Vietnam__nan,0,1,0,0,0,0,...,1,0,0,1,0,0,0,1,0,4.0
93797,2020-11-30,Yemen,NaN,Yemen__nan,1,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,14.0
93798,2020-11-30,Zambia,NaN,Zambia__nan,0,1,0,0,0,0,...,1,0,0,1,0,0,0,1,0,39.0


In [54]:
# Remove non USA regions for now. Comment out below line if we want every country.
oxford_data = oxford_data[(oxford_data['CountryName'] == 'United States') & (oxford_data['RegionName'].notnull())]
print(oxford_data['RegionName'].unique())

['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'Florida' 'Georgia' 'Hawaii' 'Idaho' 'Illinois'
 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Maryland'
 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana'
 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'Rhode Island' 'South Carolina' 'South Dakota' 'Tennessee' 'Texas' 'Utah'
 'Vermont' 'Virgin Islands' 'Virginia' 'Washington' 'Washington DC'
 'West Virginia' 'Wisconsin' 'Wyoming']


In [55]:
oxford_data

,Date,CountryName,RegionName,GeoID,C1_School closing_0.0,C1_School closing_1.0,C1_School closing_2.0,C1_School closing_3.0,C2_Workplace closing_0.0,C2_Workplace closing_1.0,...,H2_Testing policy_3.0,H3_Contact tracing_0.0,H3_Contact tracing_1.0,H3_Contact tracing_2.0,H6_Facial Coverings_0.0,H6_Facial Coverings_1.0,H6_Facial Coverings_2.0,H6_Facial Coverings_3.0,H6_Facial Coverings_4.0,NewCases
219,2020-01-01,United States,Alabama,United States__Alabama,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
220,2020-01-01,United States,Alaska,United States__Alaska,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
221,2020-01-01,United States,Arizona,United States__Arizona,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
222,2020-01-01,United States,Arkansas,United States__Arkansas,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
223,2020-01-01,United States,California,United States__California,1,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93786,2020-11-30,United States,Washington,United States__Washington,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,2462.0
93787,2020-11-30,United States,Washington DC,United States__Washington DC,0,1,0,0,0,0,...,1,0,0,1,0,0,0,1,0,104.0
93788,2020-11-30,United States,West Virginia,United States__West Virginia,0,0,1,0,0,1,...,1,0,0,1,0,0,0,1,0,845.0
93789,2020-11-30,United States,Wisconsin,United States__Wisconsin,0,0,1,0,0,1,...,1,0,1,0,0,0,0,0,1,2676.0


### Model Training

In [92]:
def process_data(df, nb_lookback_days, nb_lookback_months=20, test_months=2, static=False):    
    # Create training data across all regions for predicting one day ahead
    if static:
      df = add_static_data(df)
      X_cols = cases_col + npi_cols + census_cols + lodes_cols
    else:
      X_cols = cases_col + npi_cols      
    y_cols = ['GeoID', 'Date', 'NewCases'] # Include GeoID and date to create graphs
    X_samples = []
    y_samples = []
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    train_date = SUBMISSION_DATE-np.timedelta64((nb_lookback_months+test_months)*30, 'D')
    geo_ids = df.GeoID.unique()
    for g in geo_ids:
        gdf = df[(df.GeoID == g) & (df.Date >= train_date)]
        all_case_data = np.array(gdf[cases_col])
        all_npi_data = np.array(gdf[npi_cols])
        
        if static:
          all_census_data = np.array(gdf[census_cols])
          all_lodes_data = np.array(gdf[lodes_cols])

        all_bookkeeping_data = np.array(gdf[y_cols])

        # Create one sample for each day where we have enough data
        # Each sample consists of cases and npis for previous nb_lookback_days
        nb_total_days = len(gdf)
        
        for d in range(nb_lookback_days, nb_total_days - 1):
            X_cases = all_case_data[d-nb_lookback_days:d]
            X_npis = all_npi_data[d - nb_lookback_days:d]

            # Flatten all input data
            if static:
              X_census = all_census_data
              X_lodes = all_lodes_data
              X_sample = np.concatenate([X_cases.flatten(),
                                         X_npis.flatten(),
                                         X_census.flatten(), X_lodes.flatten()])
            else:
               X_sample = np.concatenate([X_cases.flatten(),X_npis.flatten()])              
            y_sample = all_bookkeeping_data[d]
            
            if d < nb_total_days-test_months*30:
                X_train.append(X_sample)
                y_train.append(y_sample)
            else:
                X_test.append(X_sample)
                y_test.append(y_sample)
    
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)[:,-1]
    y_test = np.array(y_test)
    return X_train, X_test, y_train, y_test

In [93]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))
def add_static_data(df):
  for col in census_cols:
    df[col] = 0
  for idx, census_row in census_data.iterrows():
    for num, feat in enumerate(census_cols):
      df[feat] = np.where(df['RegionName'] == census_row[0], census_row[num + 1] , df[feat])

  for col in lodes_cols:
    df[col] = 0
  for idx, lodes_row in lodes_data.iterrows():
    for num, feat in enumerate(lodes_cols):
      df[feat] = np.where(df['RegionName'] == lodes_row[0], lodes_row[num + 1] , df[feat])    

  return df


In [94]:
def run(df, nb_lookback_days, nb_lookback_months, test_months, model, inspect=False, graph=False, static=False):
    df_copy = df.copy()
    X_train, X_test, y_train, y_test = process_data(df_copy,
                                                    nb_lookback_days, 
                                                    nb_lookback_months,
                                                    test_months,
                                                    static
                                                   )
    model.fit(X_train, y_train)
    # Evaluate model
    train_preds = model.predict(X_train)
    train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
    print('Train MAE:', mae(train_preds, y_train))
    test_preds = model.predict(X_test)
    test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
    print('Test MAE:', mae(test_preds, y_test[:,-1]))
    
    # Inspect the learned feature coefficients for the model
    # to see what features it's paying attention to.
    if inspect:
        # Give names to the features
        x_col_names = []
        for d in range(-nb_lookback_days, 0):
            x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
        for d in range(-nb_lookback_days, 1):
            for col_name in npi_cols:
                x_col_names.append('Day ' + str(d) + ' ' + col_name)

        # View non-zero coefficients
        for (col, coeff) in zip(x_col_names, list(model.coef_)):
            if coeff != 0.:
                print(col, coeff)
        print('Intercept', model.intercept_)
        
    if graph:
        graph_np = np.hstack((y_test, test_preds.reshape(-1,1)))
        geo_ids = np.unique(graph_np[:,0])
        for geo_id in geo_ids:
            gnp = graph_np[graph_np[:,0]==geo_id, :]
            plt.figure()
            plt.plot(gnp[:,1], gnp[:,3], label='Prediction')
            plt.plot(gnp[:,1], gnp[:,2], label='Actual')
            plt.title(geo_id)
            plt.legend()
            plt.show()
            
    return model


In [82]:
# model = run(oxford_data, nb_lookback_days=30, nb_lookback_months=20, test_months=2, model=LinearRegression(), inspect=False, graph=False, static=False)

Train MAE: 149.2456828512614
Test MAE: 558.4903385799277


In [ ]:
model = run(oxford_data, 30, 20, 2, LinearRegression(), False, False, True)

In [21]:
# model = run(oxford_data, 30, 20, 2, LinearRegression(), False, False, True)

In [22]:
# model1 = run(oxford_data, 30, 20, 2, SVR(), False, False, False)

In [23]:
# Takes more than a few minutes to fit.
# model = run(oxford_data, 30, 20, 2, SVR(), False, False, True)

In [24]:
# model = run(oxford_data, 30, 20, 2, GradientBoostingRegressor(), False, False, False)

In [25]:
# Takes more than a few minutes to fit.
# model = run(oxford_data, 30, 20, 2, GradientBoostingRegressor(), False, False, True)

In [26]:
# model = run(oxford_data, 14, 20, 2, LinearRegression(), False, False, False)

In [27]:
# model = run(oxford_data, 14, 20, 2, LinearRegression(), False, False, True)

In [28]:
# model = run(oxford_data, 14, 5, 3, LinearRegression(), False, False, False)

In [29]:
# model = run(oxford_data, 14, 5, 3, LinearRegression(), False, False, True)

In [ ]:

# Save model to file (this just stores whatever is the last uncommented model)
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data. Make changes in predict.py since that's what is called

In [ ]:
%%time
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=False)

In [32]:
# Check the predictions
preds_df[preds_df['CountryName']=='United States'].tail()

,CountryName,RegionName,Date,PredictedDailyNewCases
70109,United States,Wyoming,2020-08-27,63.195753
70110,United States,Wyoming,2020-08-28,0.000000
70111,United States,Wyoming,2020-08-29,9.671481
70112,United States,Wyoming,2020-08-30,0.000000
70113,United States,Wyoming,2020-08-31,9.078761


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [33]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...

Predicting for Aruba__nan
2020-08-01: 0
2020-08-02: 26.42290735244751
2020-08-03: 35.04539155960083
2020-08-04: 0

Predicting for Afghanistan__nan
2020-08-01: 569.9594585895538
2020-08-02: 679.4045283794403
2020-08-03: 809.6152164936066
2020-08-04: 885.8713324069977

Predicting for Angola__nan
2020-08-01: 581543503.9265323
2020-08-02: 723578993.5160394
2020-08-03: 863903623.5101342
2020-08-04: 952079207.5848413

Predicting for Albania__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0

Predicting for Andorra__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0

Predicting for United Arab Emirates__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0

Predicting for Argentina__nan
2020-08-01: 2613072311.9927
2020-08-02: 3251285989.4005923
2020-08-03: 3881813732.10741
2020-08-04: 4278013187.821853

Predicting for Australia__nan
2020-08-01: 125668232.66886187
2020-08-02: 0
2020-08-03: 0
2020-08-04: 247345508

In [34]:
!head predictions/2020-08-01_2020-08-04.csv


2020-08-03: 37.18623208999634
2020-08-04: 95.31459093093872

Predicting for Chile__nan
2020-08-01: 5385422598.188316
2020-08-02: 6388194089.719509
2020-08-03: 7722902369.185822
2020-08-04: 8681543240.825226

Predicting for China__nan
2020-08-01: 5385420544.803461
2020-08-02: 6700752873.6563225
2020-08-03: 8000242970.351706
2020-08-04: 8816792768.05627

Predicting for Cote d'Ivoire__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0

Predicting for Cameroon__nan
2020-08-01: 2613066200.004764
2020-08-02: 3251280313.360809
2020-08-03: 3881807532.528441
2020-08-04: 4278006747.7288804

Predicting for Democratic Republic of Congo__nan
2020-08-01: 2613067023.3641953
2020-08-02: 3251281240.229219
2020-08-03: 3881808491.253575
2020-08-04: 4278007690.0402617

Predicting for Congo__nan
2020-08-01: 2613067065.617162
2020-08-02: 3251281246.191358
2020-08-03: 3881808523.1855197
2020-08-04: 4278007871.164787

Predicting for Colombia__nan
2020-08-01: 2613076124.718371
2020-08-02: 3251289978.

2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0

Aruba,,2020-08-01,0.0
Aruba,,2020-08-02,26.42290735244751
Aruba,,2020-08-03,35.04539155960083Predicting for Syria__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0

Aruba,,2020-08-04,0.0
Afghanistan,,2020-08-01,569.9594585895538
Afghanistan,,2020-08-02,679.4045283794403
Afghanistan,,2020-08-03,809.6152164936066
Afghanistan,,2020-08-04,885.8713324069977
Angola,,2020-08-01,581543503.9265323
2020-08-04: 0

Predicting for Chad__nan
2020-08-01: 2613066921.722326
2020-08-02: 3396845639.938886
2020-08-03: 3951367272.0418906
2020-08-04: 4165179292.082481

Predicting for Togo__nan
2020-08-01: 2613066980.997955
2020-08-02: 3251281110.356132
2020-08-03: 4027373020.3591037
2020-08-04: 4347566623.533035

Predicting for Thailand__nan
2020-08-01: 35.649115800857544
2020-08-02: 40.26691174507141
2020-08-03: 46.7828848361969
2020-08-04: 52.20852494239807

Predicting for Tajikistan__nan
2020-08-01: 203.82182908058167
2020-08-02: 0
2020-08-03: 93.1775

# Test cases
We can generate a prediction file. Let's validate a few cases...

In [35]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

2020-08-03: 147.10282063484192
2020-08-04: 259.420462846756

Predicting for United States__Florida
2020-08-01: 9855.642873048782
2020-08-02: 9376.462098360062
2020-08-03: 8060.184691667557
2020-08-04: 7320.610776185989

Predicting for United States__Georgia
2020-08-01: 3641.8313164711
2020-08-02: 3030.5714077949524
2020-08-03: 3284.638205051422
2020-08-04: 3161.952238559723

Predicting for United States__Hawaii
2020-08-01: 208.88931584358215
2020-08-02: 243.0637562274933
2020-08-03: 255.8492214679718
2020-08-04: 337.35820508003235

Predicting for United States__Iowa
2020-08-01: 435.1701729297638
2020-08-02: 526.8799097537994
2020-08-03: 374.1581394672394
2020-08-04: 288.4074490070343

Predicting for United States__Idaho
2020-08-01: 546.8400750160217
2020-08-02: 439.95403814315796
2020-08-03: 546.057831287384
2020-08-04: 410.8932099342346

Predicting for United States__Illinois
2020-08-01: 1570.474690914154
2020-08-02: 1279.11377286911
2020-08-03: 1320.7741303443909
2020-08-04: 1359.679

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [36]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...

Predicting for Aruba__nan
2020-08-01: 0
2020-08-02: 26.42290735244751
2020-08-03: 35.04539155960083
2020-08-04: 0

Predicting for Afghanistan__nan
2020-08-01: 569.9594585895538
2020-08-02: 679.4045283794403
2020-08-03: 809.6152164936066
2020-08-04: 885.8713324069977

Predicting for Angola__nan
2020-08-01: 581543503.9265323
2020-08-02: 723578993.5160394
2020-08-03: 863903623.5101342
2020-08-04: 952079207.5848413

Predicting for Albania__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0

Predicting for Andorra__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0

Predicting for United Arab Emirates__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0

Predicting for Argentina__nan
2020-08-01: 2613072311.9927
2020-08-02: 3251285989.4005923
2020-08-03: 3881813732.10741
2020-08-04: 4278013187.821853

Predicting for Australia__nan
2020-08-01: 125668232.66886187
2020-08-02: 0
2020-08-03: 0
2020-08-04: 247345508

Predicting for Kosovo__nan
2020-08-01: 3406373535.2239957
2020-08-02: 3518324739.5979514
2020-08-03: 3804367928.39319
2020-08-04: 2851501395.366613

Predicting for Romania__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0

Predicting for Russia__nan
2020-08-01: 2484783092.466788
2020-08-02: 2867275201.2475324
2020-08-03: 3273522761.1711583
2020-08-04: 3171902812.168307

Predicting for Rwanda__nan
2020-08-01: 3648504199.752153
2020-08-02: 5243800048.089054
2020-08-03: 5239844846.686808
2020-08-04: 5291960674.439743

Predicting for Saudi Arabia__nan
2020-08-01: 2613068549.620271
2020-08-02: 3251282733.6864886
2020-08-03: 3881809956.571751
2020-08-04: 4278009117.486056

Predicting for Sudan__nan
2020-08-01: 2121870059.5700097
2020-08-02: 2249723395.4783044
2020-08-03: 2638824864.345206
2020-08-04: 2430960925.5208125

Predicting for Senegal__nan
2020-08-01: 1996614570.7813334
2020-08-02: 2835960267.096166
2020-08-03: 3540105800.9573016
2020-08-04: 3736642460.7532735

Predicting 

All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [37]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...

Predicting for India__nan
2020-12-19: 2613080157.9277463 - Skipped (intermediate missing daily cases)
2020-12-20: 3251293875.8100867 - Skipped (intermediate missing daily cases)
2020-12-21: 3881818698.9894986 - Skipped (intermediate missing daily cases)
2020-12-22: 4278018527.8489995 - Skipped (intermediate missing daily cases)
2020-12-23: 4930726886.17235 - Skipped (intermediate missing daily cases)
2020-12-24: 5409672053.633425 - Skipped (intermediate missing daily cases)
2020-12-25: 6391320415.379158 - Skipped (intermediate missing daily cases)
2020-12-26: 7407284349.016508 - Skipped (intermediate missing daily cases)
2020-12-27: 8440646215.565973 - Skipped (intermediate missing daily cases)
2020-12-28: 9319307174.018896 - Skipped (intermediate missing daily cases)
2020-12-29: 10079350190.768867 - Skipped (intermediate missing daily cases)
2020-12-30: 10746700560.399399 - Skipped (intermediate missing daily cases)
2020-12-31: 

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [38]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-24
End date: 2021-06-22


In [39]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [40]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-24 to 2021-06-22...

Predicting for Afghanistan__nan
2020-12-19: 244.72511219978333 - Skipped (intermediate missing daily cases)
2020-12-20: 340.5969727039337 - Skipped (intermediate missing daily cases)
2020-12-21: 357.51615357398987 - Skipped (intermediate missing daily cases)
2020-12-22: 407.1571991443634 - Skipped (intermediate missing daily cases)
2020-12-23: 383.26021695137024 - Skipped (intermediate missing daily cases)
2020-12-24: 344.55690693855286
2020-12-25: 380.4193661212921
2020-12-26: 487.1618492603302
2020-12-27: 570.0307524204254
2020-12-28: 629.6841928958893
2020-12-29: 684.8866751194
2020-12-30: 671.5655605792999
2020-12-31: 671.3425333499908
2021-01-01: 696.7543051242828
2021-01-02: 794.5515644550323
2021-01-03: 899.277263879776
2021-01-04: 958.2146847248077
2021-01-05: 1022.2788507938385
2021-01-06: 1044.2638523578644
2021-01-07: 1028.9728138446808
2021-01-08: 1078.9484903812408
2021-01-09: 1190.616546869278
2021-01-10: 1265.33150

2021-01-04: 210.4515278339386
2021-01-05: 220.6750671863556
2021-01-06: 227.0365002155304
2021-01-07: 238.7374107837677
2021-01-08: 254.51889538764954
2021-01-09: 271.78142857551575
2021-01-10: 284.541535615921
2021-01-11: 298.7710335254669
2021-01-12: 309.8066055774689
2021-01-13: 317.3934624195099
2021-01-14: 325.530814409256
2021-01-15: 339.7037537097931
2021-01-16: 356.556880235672
2021-01-17: 370.1398174762726
2021-01-18: 382.5604946613312
2021-01-19: 391.5861256122589
2021-01-20: 401.2617371082306
2021-01-21: 410.7585232257843
2021-01-22: 422.8899586200714
2021-01-23: 435.98605847358704
2021-01-24: 449.9021236896515
2021-01-25: 461.2969868183136
2021-01-26: 469.7495300769806
2021-01-27: 477.6114857196808
2021-01-28: 486.9796345233917
2021-01-29: 497.21654057502747
2021-01-30: 508.47317814826965
2021-01-31: 519.9563591480255
2021-02-01: 530.4540646076202
2021-02-02: 538.2097294330597
2021-02-03: 544.7111465930939
2021-02-04: 552.4230625629425
2021-02-05: 561.7318470478058
2021-02-

2021-05-22: 17068.37449479103
2021-05-23: 17068.720071554184
2021-05-24: 17069.99143266678
2021-05-25: 17070.392891168594
2021-05-26: 17068.75264120102
2021-05-27: 17065.392227888107
2021-05-28: 17061.902590990067
2021-05-29: 17059.83285331726
2021-05-30: 17059.542844057083
2021-05-31: 17060.116892814636
2021-06-01: 17060.06657934189
2021-06-02: 17058.35443663597
2021-06-03: 17055.19616127014
2021-06-04: 17051.894132375717
2021-06-05: 17049.73321413994
2021-06-06: 17049.072090625763
2021-06-07: 17049.211771965027
2021-06-08: 17048.909877300262
2021-06-09: 17047.25349545479
2021-06-10: 17044.39479136467
2021-06-11: 17041.373718976974
2021-06-12: 17039.26534485817
2021-06-13: 17038.425101995468
2021-06-14: 17038.314942121506
2021-06-15: 17037.890042304993
2021-06-16: 17036.37583374977
2021-06-17: 17033.857609272003
2021-06-18: 17031.164549827576
2021-06-19: 17029.20153903961
2021-06-20: 17028.308678388596
2021-06-21: 17028.06758904457
2021-06-22: 17027.613746881485

Predicting for Bosnia

2021-02-09: 0
2021-02-10: 0
2021-02-11: 0
2021-02-12: 0
2021-02-13: 0
2021-02-14: 0
2021-02-15: 0
2021-02-16: 0
2021-02-17: 0
2021-02-18: 0
2021-02-19: 0
2021-02-20: 0
2021-02-21: 0
2021-02-22: 0
2021-02-23: 0
2021-02-24: 0
2021-02-25: 0
2021-02-26: 0
2021-02-27: 0
2021-02-28: 0
2021-03-01: 0
2021-03-02: 0
2021-03-03: 0
2021-03-04: 0
2021-03-05: 0
2021-03-06: 0
2021-03-07: 0
2021-03-08: 0
2021-03-09: 0
2021-03-10: 0
2021-03-11: 0
2021-03-12: 0
2021-03-13: 0
2021-03-14: 0
2021-03-15: 0
2021-03-16: 0
2021-03-17: 0
2021-03-18: 0
2021-03-19: 0
2021-03-20: 0
2021-03-21: 0
2021-03-22: 0
2021-03-23: 0
2021-03-24: 0
2021-03-25: 0
2021-03-26: 0
2021-03-27: 0
2021-03-28: 0
2021-03-29: 0
2021-03-30: 0
2021-03-31: 0
2021-04-01: 0
2021-04-02: 0
2021-04-03: 0
2021-04-04: 0
2021-04-05: 0
2021-04-06: 0
2021-04-07: 0
2021-04-08: 0
2021-04-09: 0
2021-04-10: 0
2021-04-11: 0
2021-04-12: 0
2021-04-13: 0
2021-04-14: 0
2021-04-15: 0
2021-04-16: 0
2021-04-17: 0
2021-04-18: 0
2021-04-19: 0
2021-04-20: 0
2021-0


Predicting for Brazil__Ceara
2020-12-19: 440.2304005622864 - Skipped (intermediate missing daily cases)
2020-12-20: 1035.6774725914001 - Skipped (intermediate missing daily cases)
2020-12-21: 967.9728646278381 - Skipped (intermediate missing daily cases)
2020-12-22: 2160.087662220001 - Skipped (intermediate missing daily cases)
2020-12-23: 2624.5720086097717 - Skipped (intermediate missing daily cases)
2020-12-24: 2376.6837153434753
2020-12-25: 1832.0837426185608
2020-12-26: 1477.8629202842712
2020-12-27: 1523.0273022651672
2020-12-28: 2150.877203464508
2020-12-29: 3138.870129108429
2020-12-30: 3121.7526268959045
2020-12-31: 3130.0210671424866
2021-01-01: 2850.786225795746
2021-01-02: 2632.6919569969177
2021-01-03: 2533.115948200226
2021-01-04: 3398.2029843330383
2021-01-05: 4135.707327365875
2021-01-06: 4721.7376618385315
2021-01-07: 4248.931889057159
2021-01-08: 4052.806433200836
2021-01-09: 4141.524129390717
2021-01-10: 4124.5578789711
2021-01-11: 4458.721412181854
2021-01-12: 5348

...

2021-03-10: 1134.8337662220001
2021-03-11: 1134.7372887134552
2021-03-12: 1138.1903946399689
2021-03-13: 1154.7541325092316
2021-03-14: 1175.3076536655426
2021-03-15: 1192.3416550159454
2021-03-16: 1200.8977167606354
2021-03-17: 1202.7890121936798
2021-03-18: 1201.8115541934967
2021-03-19: 1205.9714705944061
2021-03-20: 1217.1091072559357
2021-03-21: 1234.4408705234528
2021-03-22: 1249.8584988117218
2021-03-23: 1255.8270618915558
2021-03-24: 1257.0654113292694
2021-03-25: 1257.370585680008
2021-03-26: 1259.210423707962
2021-03-27: 1267.8723304271698
2021-03-28: 1282.8358643054962
2021-03-29: 1294.3468716144562
2021-03-30: 1299.7232954502106
2021-03-31: 1301.304661989212
2021-04-01: 1300.188004732132
2021-04-02: 1301.1443445682526
2021-04-03: 1308.977290391922
2021-04-04: 1319.81436085701
2021-04-05: 1329.0810272693634
2021-04-06: 1334.3619759082794
2021-04-07: 1335.0556581020355
2021-04-08: 1333.3659317493439
2021-04-09: 1334.3317935466766
2021-04-10: 1339.9071424007416
2021-04-11: 134

2021-04-01: 0
2021-04-02: 0
2021-04-03: 0
2021-04-04: 0
2021-04-05: 0
2021-04-06: 0
2021-04-07: 0
2021-04-08: 0
2021-04-09: 0
2021-04-10: 0
2021-04-11: 0
2021-04-12: 0
2021-04-13: 0
2021-04-14: 0
2021-04-15: 0
2021-04-16: 0
2021-04-17: 0
2021-04-18: 0
2021-04-19: 0
2021-04-20: 0
2021-04-21: 0
2021-04-22: 0
2021-04-23: 0
2021-04-24: 0
2021-04-25: 0
2021-04-26: 0
2021-04-27: 0
2021-04-28: 0
2021-04-29: 0
2021-04-30: 0
2021-05-01: 0
2021-05-02: 0
2021-05-03: 0
2021-05-04: 0
2021-05-05: 0
2021-05-06: 0
2021-05-07: 0
2021-05-08: 0
2021-05-09: 0
2021-05-10: 0
2021-05-11: 0
2021-05-12: 0
2021-05-13: 0
2021-05-14: 0
2021-05-15: 0
2021-05-16: 0
2021-05-17: 0
2021-05-18: 0
2021-05-19: 0
2021-05-20: 0
2021-05-21: 0
2021-05-22: 0
2021-05-23: 0
2021-05-24: 0
2021-05-25: 0
2021-05-26: 0
2021-05-27: 0
2021-05-28: 0
2021-05-29: 0
2021-05-30: 0
2021-05-31: 0
2021-06-01: 0
2021-06-02: 0
2021-06-03: 0
2021-06-04: 0
2021-06-05: 0
2021-06-06: 0
2021-06-07: 0
2021-06-08: 0
2021-06-09: 0
2021-06-10: 0
2021-0

2021-04-15: 711.2557842731476
2021-04-16: 710.390506029129
2021-04-17: 711.2066900730133
2021-04-18: 713.2173750400543
2021-04-19: 715.0897018909454
2021-04-20: 715.9672148227692
2021-04-21: 715.5488016605377
2021-04-22: 714.2921802997589
2021-04-23: 713.5038273334503
2021-04-24: 714.0400593280792
2021-04-25: 715.5396807193756
2021-04-26: 717.0723822116852
2021-04-27: 717.8106243610382
2021-04-28: 717.385701417923
2021-04-29: 716.2949955463409
2021-04-30: 715.5587751865387
2021-05-01: 715.8626663684845
2021-05-02: 717.0100567340851
2021-05-03: 718.2501842975616
2021-05-04: 718.8353798389435
2021-05-05: 718.471039056778
2021-05-06: 717.5150110721588
2021-05-07: 716.814225435257
2021-05-08: 716.9782650470734
2021-05-09: 717.8515484333038
2021-05-10: 718.8321497440338
2021-05-11: 719.318806886673
2021-05-12: 719.0017387866974
2021-05-13: 718.1600887775421
2021-05-14: 717.526778459549
2021-05-15: 717.5815660953522
2021-05-16: 718.2314646244049
2021-05-17: 719.0258486270905
2021-05-18: 719.

2021-01-16: 1759.84770321846
2021-01-17: 1793.4900887012482
2021-01-18: 1821.0036747455597
2021-01-19: 1851.2777149677277
2021-01-20: 1915.9614593982697
2021-01-21: 1967.3112480640411
2021-01-22: 2054.9546806812286
2021-01-23: 2122.151368379593
2021-01-24: 2157.7584393024445
2021-01-25: 2176.5897648334503
2021-01-26: 2213.8912346363068
2021-01-27: 2257.940171480179
2021-01-28: 2311.0907833576202
2021-01-29: 2377.8881618976593
2021-01-30: 2431.370564699173
2021-01-31: 2469.5102875232697
2021-02-01: 2494.048812150955
2021-02-02: 2521.3633835315704
2021-02-03: 2552.9357788562775
2021-02-04: 2603.292102098465
2021-02-05: 2658.4629662036896
2021-02-06: 2708.592847108841
2021-02-07: 2737.81112408638
2021-02-08: 2761.872481584549
2021-02-09: 2781.4248325824738
2021-02-10: 2810.397950410843
2021-02-11: 2847.768414735794
2021-02-12: 2897.2295200824738
2021-02-13: 2936.7028362751007
2021-02-14: 2961.122128725052
2021-02-15: 2979.1896736621857
2021-02-16: 2997.510804414749
2021-02-17: 3019.727681

2021-04-01: 15187.933657884598
2021-04-02: 15208.57586312294
2021-04-03: 15232.784790277481
2021-04-04: 15258.298293352127
2021-04-05: 15280.899080514908
2021-04-06: 15298.527798891068
2021-04-07: 15311.882403612137
2021-04-08: 15324.0421936512
2021-04-09: 15337.999098062515
2021-04-10: 15354.997741937637
2021-04-11: 15373.133659601212
2021-04-12: 15389.251479387283
2021-04-13: 15401.201126337051
2021-04-14: 15409.6903860569
2021-04-15: 15417.102385759354
2021-04-16: 15425.92622590065
2021-04-17: 15437.357180833817
2021-04-18: 15450.014683961868
2021-04-19: 15461.049466371536
2021-04-20: 15468.781221628189
2021-04-21: 15473.713737726212
2021-04-22: 15477.592669725418
2021-04-23: 15482.670805215836
2021-04-24: 15490.020323038101
2021-04-25: 15498.46170258522
2021-04-26: 15505.722059488297
2021-04-27: 15510.445935487747
2021-04-28: 15512.733942270279
2021-04-29: 15514.134484529495
2021-04-30: 15516.574410676956
2021-05-01: 15520.915047883987
2021-05-02: 15526.241029977798
2021-05-03: 155